In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import json

# Utilities and Functions

## BLEU Calculation

In [3]:
from sacrebleu import BLEU
from evaluate import load

In [4]:
import warnings
import logging
import os
from transformers import logging as transformers_logging

# Suppress ALL HuggingFace warnings
transformers_logging.set_verbosity_error()
warnings.filterwarnings("ignore")  # This suppresses all warnings

# Suppress NLTK messages - try both methods
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
def calculate_metrics_batch(references,hypotheses,verbose= False):
    # Load metrics
    bleu = load('bleu')
    meteor = load("meteor")
    rouge = load("rouge")
    bertscore = load("bertscore")
    
    # Format references for metrics that expect a single reference
    # this will transform references from:
    # [[ref1],[re2],[ref3]] to [ref1,ref2,ref3]
    single_refs = [ref_list[0] for ref_list in references]
    
    
    # Calculate BLEU
    bleu_score = bleu.compute(predictions=hypotheses, references=single_refs)
    
    # Calculate METEOR
    meteor_score = meteor.compute(
        predictions=hypotheses,
        references=single_refs
    )
    
    # Calculate ROUGE
    rouge_scores = rouge.compute(
        predictions=hypotheses,
        references=single_refs,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    
    bert_scores = bertscore.compute(
        predictions=hypotheses,
        references=single_refs,
        lang="ar",  # Set language to Arabic
        # model_type="aubmindlab/bert-large-arabertv02"  # Using ArabertV2, a specialized Arabic BERT model
        # Alternative models:
        # - "bert-base-multilingual-cased"  # If you want a multilingual model
        # - "CAMeL-Lab/bert-base-arabic-camelbert-da"  # Another good Arabic BERT
    )
    
    # Average BERTScore across all sentences
    bert_f1 = sum(bert_scores['f1']) / len(bert_scores['f1'])
    
    if verbose:
        # Return all scores
        return {
            'bleu': bleu_score,
            'meteor': meteor_score['meteor'],
            'rouge': {
                'rouge1': {
                    'precision': rouge_scores['rouge1_precision'],
                    'recall': rouge_scores['rouge1_recall'],
                    'f1': rouge_scores['rouge1']
                },
                'rouge2': {
                    'precision': rouge_scores['rouge2_precision'],
                    'recall': rouge_scores['rouge2_recall'],
                    'f1': rouge_scores['rouge2']
                },
                'rougeL': {
                    'precision': rouge_scores['rougeL_precision'],
                    'recall': rouge_scores['rougeL_recall'],
                    'f1': rouge_scores['rougeL']
                }
            },
            'bert_score': {
                'precision': sum(bert_scores['precision']) / len(bert_scores['precision']),
                'recall': sum(bert_scores['recall']) / len(bert_scores['recall']),
                'f1': bert_f1
            }
        }
    else:
        # Return only primary metrics commonly reported in literature
        return {
            'bleu': bleu_score['bleu'],
            'meteor': meteor_score['meteor'],
            'rouge_l_f1': rouge_scores['rougeL'],  # Most commonly reported ROUGE metric
            'bert_score_f1': bert_f1  # Most commonly reported BERTScore metric
        }

In [6]:
def read_jsonl(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [7]:
def is_valid_post(post_dict):
    generated_post = post_dict['generated_post']
    # filter if it contains the start or end tokens:
    if "<START>" in generated_post or "<END>" in generated_post:
        return False
    # filter by length, set minimum to 50 words
    if len([word for word in generated_post.split() if word]) < 50:
        return False
    return True

In [8]:
def remove_posts_duplicates(posts):
    unique_posts = []
    for post in posts:
        if post['original_post'] in [unique_post['original_post'] for unique_post in unique_posts]:
            continue
        unique_posts.append(post)
    return unique_posts

In [9]:
def get_common_posts(posts):
    # Create a set of original_posts that exist in all models
    common_originals = {}
    for model_posts in posts.values():
        original_posts = {post['original_post'] for post in model_posts}
        if not common_originals:
            common_originals = set(original_posts)
        else:
            common_originals = set.intersection(common_originals, original_posts)
    
    # Find intersection of all sets to get common original posts
    # common_originals = set.intersection(*original_posts_sets)
    
    # Filter posts to only include those with common original posts
    filtered_posts = {}
    for model_name, model_posts in posts.items():
        filtered_posts[model_name] = [
            post for post in model_posts 
            if post['original_post'] in common_originals
        ]
    
    return filtered_posts


## Descriptive statistics calculation

In [10]:
import re
import pandas as pd
import numpy as np

def get_text_statistics(text):
    """Calculate word, sentence, and sentence length statistics for a given text."""
    # Clean text
    text = text.strip()
    
    # Split into sentences (handle multiple punctuation marks)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    
    # Get word counts for each sentence
    sentence_word_counts = [len(s.split()) for s in sentences]
    
    stats = {
        'word_count': sum(sentence_word_counts),
        'sentence_count': len(sentences),
        'avg_sentence_length': np.mean(sentence_word_counts) if sentence_word_counts else 0,
        'min_sentence_length': min(sentence_word_counts) if sentence_word_counts else 0,
        'max_sentence_length': max(sentence_word_counts) if sentence_word_counts else 0
    }
    
    return stats

def calculate_descriptive_statistics(filtered_posts):
    """Analyze and compare original and generated posts across models."""
    results = {}
    
    # First, analyze original posts (should be same across models)
    original_stats = []
    first_model = next(iter(filtered_posts.values()))
    for item in first_model:
        original_stats.append(get_text_statistics(item['original_post']))
    
    # Calculate original post statistics
    orig_df = pd.DataFrame(original_stats)
    results['original'] = {
        'word_count': {
            'min': orig_df['word_count'].min(),
            'max': orig_df['word_count'].max(),
            'avg': orig_df['word_count'].mean()
        },
        'sentence_count': {
            'min': orig_df['sentence_count'].min(),
            'max': orig_df['sentence_count'].max(),
            'avg': orig_df['sentence_count'].mean()
        },
        'sentence_length': {
            'min': orig_df['min_sentence_length'].min(),
            'max': orig_df['max_sentence_length'].max(),
            'avg': orig_df['avg_sentence_length'].mean()
        }
    }
    
    # Analyze generated posts for each model
    results['generated'] = {}
    for model_name, posts in filtered_posts.items():
        generated_stats = []
        for item in posts:
            generated_stats.append(get_text_statistics(item['generated_post']))
        
        gen_df = pd.DataFrame(generated_stats)
        results['generated'][model_name] = {
            'word_count': {
                'min': gen_df['word_count'].min(),
                'max': gen_df['word_count'].max(),
                'avg': gen_df['word_count'].mean()
            },
            'sentence_count': {
                'min': gen_df['sentence_count'].min(),
                'max': gen_df['sentence_count'].max(),
                'avg': gen_df['sentence_count'].mean()
            },
            'sentence_length': {
                'min': gen_df['min_sentence_length'].min(),
                'max': gen_df['max_sentence_length'].max(),
                'avg': gen_df['avg_sentence_length'].mean()
            }
        }
    
    return results

def print_discriptive_statistics_comparision(results):
    """Print a formatted comparison of the analysis results."""
    # Print original post statistics
    print("Original Posts Statistics:")
    print("=" * 60)
    orig_stats = results['original']
    
    for metric, values in orig_stats.items():
        print(f"\n{metric.replace('_', ' ').title()}:")
        print(f"  Min: {values['min']:.1f}")
        print(f"  Max: {values['max']:.1f}")
        print(f"  Avg: {values['avg']:.1f}")
    
    # Print generated post statistics for each model
    print("\nGenerated Posts Statistics:")
    print("=" * 60)
    
    for model_name, stats in results['generated'].items():
        print(f"\n{model_name}:")
        print("-" * 40)
        
        for metric, values in stats.items():
            print(f"\n{metric.replace('_', ' ').title()}:")
            orig_values = orig_stats[metric]
            
            print(f"  Min: {values['min']:.1f} (Δ: {values['min'] - orig_values['min']:+.1f})")
            print(f"  Max: {values['max']:.1f} (Δ: {values['max'] - orig_values['max']:+.1f})")
            print(f"  Avg: {values['avg']:.1f} (Δ: {values['avg'] - orig_values['avg']:+.1f})")

In [11]:
# save posts list as jsonl file
def save_jsonl(posts, filename):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            for post in posts:
                file.write(json.dumps(post, ensure_ascii=False) + '\n')
    except IOError as e:
        print(f"Error writing file: {e}")
    except json.JSONEncodeError as e:
        print(f"Error encoding JSON: {e}")

# Polished posts

In [12]:
posts = dict(
    allam = read_jsonl('generated_arabic_datasets/allam/arabic_social_media_dataset/by_polishing_posts_generation.jsonl'),
    openai= read_jsonl('generated_arabic_datasets/openai/arabic_social_media_dataset/by_polishing_posts_generation.jsonl'),
    # claude= json.load(open('generated_arabic_datasets/claude/arabic_social_media_dataset/by_polishing_posts_generation.jsonl', 'r')),
    jais = read_jsonl('generated_arabic_datasets/jais-batched/arabic_social_media_dataset/by_polishing_posts_generation.jsonl'),
    llama= read_jsonl('generated_arabic_datasets/llama-batched/arabic_social_media_dataset/by_polishing_posts_generation.jsonl'),
)

In [13]:
len(posts['allam']), len(posts['jais']), len(posts['llama']), len(posts['openai'])

(3500, 3500, 3500, 3500)

filter posts

In [14]:
filtered_posts = {}

In [15]:
for model_name, model_posts in posts.items():
    filtered_posts[model_name] = remove_posts_duplicates(model_posts)
len(filtered_posts['allam']), len(filtered_posts['jais']), len(filtered_posts['llama']), len(filtered_posts['openai'])

(3484, 3484, 3484, 3484)

In [16]:
for model_name, posts in filtered_posts.items():
    filtered_posts[model_name] = []
    for post in posts:
        if is_valid_post(post):
            filtered_posts[model_name].append(post)
len(filtered_posts['allam']), len(filtered_posts['jais']), len(filtered_posts['llama']), len(filtered_posts['openai'])

(3434, 3366, 3478, 3482)

In [17]:
filtered_posts = get_common_posts(filtered_posts)

In [18]:
len(filtered_posts['allam']), len(filtered_posts['jais']), len(filtered_posts['llama'])

(3318, 3318, 3318)

In [19]:
sorted([post['original_post'] for post in filtered_posts['allam']])[:5],sorted([post['original_post'] for post in filtered_posts['allam']])[-5:]

([' ? يجوز أن تلعني أي شيء يسقط من السماء حتي لو كان مطرا . . فقط عندما تكون خبازا ماهرا ، فإنك تصنع العجين الذي تريد و ببراعة تضعه في قوالب تبرز تحفة صنيعك ... إليف خبازة ماهرة ، تصنع في هذة الرواية كما في غيرها ، عجين رائع من الفلسفة ، التاريخ ، المشاعر الإنسانية ، الطبائع الاجتماعية ، و تعرض وجهة النظر و ضدها ، الحقيقة و الزيف و كل شيء ، و تبرز هذا العجين المتماسك في قالب روائي محبوك .. * القصة :- ( الإنسان بطل قصة حياته ؛ كل ما يقابله و يعيشه لأجله ، و كل من يقابلهم كذلك ... يلعب الجميع في قصتك دورا ، دور بطولة كان او دورا فرعيا ... و انت كذلك تلعب دوراً في قصص الاخرين ..حيث تتداخل تللك القصص الصغيرة كلها في رواية أكبر لتقص قصة الكون ...) برهنت إليف علي ذلك ببراعة مبهرة ؛ تداخلت الشخصيات جميعاً بقوالبها القصصية الصغيرة ؛ لتصنع القالب الروائي الكبير المبهر المتقن الصنع ... و طبعاً حملت الرواية مفاجأة مبهرة بما يكفي ، و عقدة عجيبة ، و مشاعر طاغية ، و عالما مختلفا ، و ثقافة مغايرة ، و قضية شائكة ... * الفكر :- لم أر أحسن من إليف ممث? لجنسها في الفكر الفلسفي ذي المسحة الدينية المطبوخ ف

In [20]:
sorted([post['original_post'] for post in filtered_posts['jais']])[:5],sorted([post['original_post'] for post in filtered_posts['jais']])[-5:]

([' ? يجوز أن تلعني أي شيء يسقط من السماء حتي لو كان مطرا . . فقط عندما تكون خبازا ماهرا ، فإنك تصنع العجين الذي تريد و ببراعة تضعه في قوالب تبرز تحفة صنيعك ... إليف خبازة ماهرة ، تصنع في هذة الرواية كما في غيرها ، عجين رائع من الفلسفة ، التاريخ ، المشاعر الإنسانية ، الطبائع الاجتماعية ، و تعرض وجهة النظر و ضدها ، الحقيقة و الزيف و كل شيء ، و تبرز هذا العجين المتماسك في قالب روائي محبوك .. * القصة :- ( الإنسان بطل قصة حياته ؛ كل ما يقابله و يعيشه لأجله ، و كل من يقابلهم كذلك ... يلعب الجميع في قصتك دورا ، دور بطولة كان او دورا فرعيا ... و انت كذلك تلعب دوراً في قصص الاخرين ..حيث تتداخل تللك القصص الصغيرة كلها في رواية أكبر لتقص قصة الكون ...) برهنت إليف علي ذلك ببراعة مبهرة ؛ تداخلت الشخصيات جميعاً بقوالبها القصصية الصغيرة ؛ لتصنع القالب الروائي الكبير المبهر المتقن الصنع ... و طبعاً حملت الرواية مفاجأة مبهرة بما يكفي ، و عقدة عجيبة ، و مشاعر طاغية ، و عالما مختلفا ، و ثقافة مغايرة ، و قضية شائكة ... * الفكر :- لم أر أحسن من إليف ممث? لجنسها في الفكر الفلسفي ذي المسحة الدينية المطبوخ ف

In [21]:
sorted([post['original_post'] for post in filtered_posts['llama']])[:5],sorted([post['original_post'] for post in filtered_posts['llama']])[-5:]

([' ? يجوز أن تلعني أي شيء يسقط من السماء حتي لو كان مطرا . . فقط عندما تكون خبازا ماهرا ، فإنك تصنع العجين الذي تريد و ببراعة تضعه في قوالب تبرز تحفة صنيعك ... إليف خبازة ماهرة ، تصنع في هذة الرواية كما في غيرها ، عجين رائع من الفلسفة ، التاريخ ، المشاعر الإنسانية ، الطبائع الاجتماعية ، و تعرض وجهة النظر و ضدها ، الحقيقة و الزيف و كل شيء ، و تبرز هذا العجين المتماسك في قالب روائي محبوك .. * القصة :- ( الإنسان بطل قصة حياته ؛ كل ما يقابله و يعيشه لأجله ، و كل من يقابلهم كذلك ... يلعب الجميع في قصتك دورا ، دور بطولة كان او دورا فرعيا ... و انت كذلك تلعب دوراً في قصص الاخرين ..حيث تتداخل تللك القصص الصغيرة كلها في رواية أكبر لتقص قصة الكون ...) برهنت إليف علي ذلك ببراعة مبهرة ؛ تداخلت الشخصيات جميعاً بقوالبها القصصية الصغيرة ؛ لتصنع القالب الروائي الكبير المبهر المتقن الصنع ... و طبعاً حملت الرواية مفاجأة مبهرة بما يكفي ، و عقدة عجيبة ، و مشاعر طاغية ، و عالما مختلفا ، و ثقافة مغايرة ، و قضية شائكة ... * الفكر :- لم أر أحسن من إليف ممث? لجنسها في الفكر الفلسفي ذي المسحة الدينية المطبوخ ف

save filetered posts

In [22]:
# save_jsonl(filtered_posts['allam'], 'generated_arabic_datasets/allam/arabic_social_media_dataset/by_polishing_posts_generation_filtered.jsonl')
# save_jsonl(filtered_posts['jais'], 'generated_arabic_datasets/jais-batched/arabic_social_media_dataset/by_polishing_posts_generation_filtered.jsonl')
# save_jsonl(filtered_posts['llama'], 'generated_arabic_datasets/llama-batched/arabic_social_media_dataset/by_polishing_posts_generation_filtered.jsonl')
# save_jsonl(filtered_posts['openai'], 'generated_arabic_datasets/openai/arabic_social_media_dataset/by_polishing_posts_generation_filtered.jsonl')

print descriptive statistics

In [23]:
print_discriptive_statistics_comparision(calculate_descriptive_statistics(filtered_posts))

Original Posts Statistics:

Word Count:
  Min: 135.0
  Max: 1546.0
  Avg: 867.4

Sentence Count:
  Min: 2.0
  Max: 357.0
  Avg: 53.6

Sentence Length:
  Min: 1.0
  Max: 824.0
  Avg: 25.7

Generated Posts Statistics:

allam:
----------------------------------------

Word Count:
  Min: 50.0 (Δ: -85.0)
  Max: 2705.0 (Δ: +1159.0)
  Avg: 627.4 (Δ: -240.1)

Sentence Count:
  Min: 1.0 (Δ: -1.0)
  Max: 287.0 (Δ: -70.0)
  Avg: 41.7 (Δ: -11.9)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 1309.0 (Δ: +485.0)
  Avg: 17.9 (Δ: -7.8)

openai:
----------------------------------------

Word Count:
  Min: 76.0 (Δ: -59.0)
  Max: 1761.0 (Δ: +215.0)
  Avg: 449.5 (Δ: -418.0)

Sentence Count:
  Min: 3.0 (Δ: +1.0)
  Max: 232.0 (Δ: -125.0)
  Avg: 29.2 (Δ: -24.4)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 211.0 (Δ: -613.0)
  Avg: 16.8 (Δ: -8.9)

jais:
----------------------------------------

Word Count:
  Min: 53.0 (Δ: -82.0)
  Max: 409.0 (Δ: -1137.0)
  Avg: 305.3 (Δ: -562.2)

Sentence Count:
  Min: 1.0 (Δ:

calclate similarity metrics

In [24]:
for model_name, posts in filtered_posts.items():
    print(
        model_name, ':',
        calculate_metrics_batch(
            references=[[post['original_post']] for post in posts],
            hypotheses=[post['generated_post'] for post in posts]
        )
    )

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


allam : {'bleu': 0.4510175287956605, 'meteor': 0.5491046138025026, 'rouge_l_f1': 0.42261546086031354, 'bert_score_f1': 0.8635566212492869}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


openai : {'bleu': 0.11303841647206576, 'meteor': 0.27175782529759274, 'rouge_l_f1': 0.3713843398911943, 'bert_score_f1': 0.8231028864691242}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


jais : {'bleu': 0.055797754660547316, 'meteor': 0.2038165413415382, 'rouge_l_f1': 0.22223327881156463, 'bert_score_f1': 0.8170978338909551}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


llama : {'bleu': 0.04241922248059797, 'meteor': 0.26557163772731207, 'rouge_l_f1': 0.2614597352267972, 'bert_score_f1': 0.9504559305218633}


In [ ]:
exit()

: 